In [1]:
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import nltk
from nltk.corpus import stopwords 
import re
analyser = SentimentIntensityAnalyzer()

In [2]:
data=pd.read_csv("dress-flipkart-final-final.csv")
data.head()
data=data[['URL','ID','ITEM','STARS','NUMBER OF RATINGS','NUMBER OF REVIEWS', 'LIST OF REVIEWS']]

In [3]:
data.rename(columns = {'ID':'id','ITEM':'desc', 'LIST OF REVIEWS':'reviews','NUMBER OF REVIEWS':'num_reviews','STARS':'stars','NUMBER OF RATINGS':'num_ratings'}, inplace=True)
data.head()

,URL,id,desc,stars,num_ratings,num_reviews,reviews
0,http://flipkart.com/kalawatis-women-fit-flare-...,0,Women Fit and Flare White Dress,4.0,4,0,[]
1,http://flipkart.com/iqraar-women-fit-flare-pur...,1,Women Fit and Flare Purple Dress,4.3,20596,2761,"['It was an awesome dress . I loved it', 'Very..."
2,http://flipkart.com/aayu-women-fit-flare-multi...,2,Women Fit and Flare Multicolor Dress,4.0,106335,15304,['So comfortable dress in pregnancy time....'...
3,http://flipkart.com/sheetal-associates-women-w...,3,Women Wrap Pink Dress,4.2,9636,1124,"[""It's absolutely stunning 🥰"", 'This dress is ..."
4,http://flipkart.com/purshottam-wala-women-maxi...,4,Women Maxi Black Dress,4.2,12918,1512,"['Awsm n comfortable', '🌝👌🏻 beautiful', ""I'm r..."


In [4]:
data.reviews = data.reviews.str.lower()
data.reviews = data.reviews.str.replace('\n','').str.replace('[\'!"#$%&\()*+,-./:;<=>?@[\\]^_`{|}~]','')

C:\Users\ARNAVJ~1\AppData\Local\Temp/ipykernel_21824/2430931611.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  data.reviews = data.reviews.str.replace('\n','').str.replace('[\'!"#$%&\()*+,-./:;<=>?@[\\]^_`{|}~]','')


In [5]:
data['vader_score'] = 1.0
for ind in data.index: 
    rev=data['reviews'][ind]
    x = rev.split()
#     print(x)
    sum_score=0.0
    for i in x:
        score = analyser.polarity_scores(i)
        sum_score=sum_score+score['compound']
    data['vader_score'][ind] =sum_score

C:\Users\ARNAVJ~1\AppData\Local\Temp/ipykernel_21824/2037979432.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['vader_score'][ind] =sum_score


In [6]:
total_star_givers = data['num_ratings'].sum()     
total_review_givers = data['num_reviews'].sum()
# print(total_star_givers)
# print(total_review_givers)

In [7]:
data['final_score'] = ((data['stars']*data['num_ratings'])/total_star_givers)+((data['vader_score']*data['num_reviews'])/total_review_givers)

In [8]:
data.head()

,URL,id,desc,stars,num_ratings,num_reviews,reviews,vader_score,final_score
0,http://flipkart.com/kalawatis-women-fit-flare-...,0,Women Fit and Flare White Dress,4.0,4,0,,0.0000,0.000007
1,http://flipkart.com/iqraar-women-fit-flare-pur...,1,Women Fit and Flare Purple Dress,4.3,20596,2761,it was an awesome dress i loved it very nice ...,3.4872,0.064224
2,http://flipkart.com/aayu-women-fit-flare-multi...,2,Women Fit and Flare Multicolor Dress,4.0,106335,15304,so comfortable dress in pregnancy time very d...,1.8009,0.253933
3,http://flipkart.com/sheetal-associates-women-w...,3,Women Wrap Pink Dress,4.2,9636,1124,its absolutely stunning 🥰 this dress is amezin...,2.3018,0.024059
4,http://flipkart.com/purshottam-wala-women-maxi...,4,Women Maxi Black Dress,4.2,12918,1512,awsm n comfortable 🌝👌🏻 beautiful im really sat...,2.5633,0.033442


In [9]:
data.to_csv('dresses_csv_final.csv')

In [18]:
dict_bigram={}
item_bigram={}
nltk.download('stopwords')

stop_words = set(stopwords.words('english')) 
for ind in data.index:
    d=data['desc'][ind][6:-5]
    d= re.sub(r'[^\w\s]', '', d) 
    nltk_tokens = nltk.word_tokenize(d)
    nltk_tokens = [w for w in nltk_tokens if not w in stop_words] 
    arr=list(nltk.bigrams(nltk_tokens))
    for i in arr:
        if i in dict_bigram:
            dict_bigram[i]=dict_bigram[i]+1
            item_bigram[" ".join(i)].append(ind)
        else:
            dict_bigram[i]=1
            item_bigram[" ".join(i)]=[]
            item_bigram[" ".join(i)].append(ind)
print(item_bigram) 

{'Fit Flare': [0, 1, 2, 11, 12, 20, 25, 26, 27, 30, 32, 35, 37, 46, 49, 54, 55, 66, 69, 74, 76, 86, 91, 101, 105, 110, 111, 113, 130, 133, 135, 137, 146, 147, 148, 150, 157, 160, 165, 168, 174, 177, 183, 186, 209, 216, 219, 223, 224, 238, 250, 261, 263, 264, 265, 267, 276, 281, 284, 292, 293, 297, 300, 302, 312, 313, 314, 323, 331, 333, 338, 342, 343, 363, 364], 'Flare White': [0, 148, 150], 'Flare Purple': [1, 37, 74, 110, 157, 160, 186, 223, 238, 261, 297, 312, 338], 'Flare Multicolor': [2, 12, 32, 66, 69, 86, 91, 147, 165, 267, 292, 300, 333], 'Wrap Pink': [3, 163, 171], 'Maxi Black': [4, 29, 65, 204, 205, 225, 254, 291, 305], 'Ethnic Dress': [5, 200, 229, 232], 'Dress Green': [5], 'Bodycon Black': [6, 18, 47, 52, 57, 62, 103, 106, 116, 136, 176, 178, 202, 207, 217, 220, 228, 249, 256, 258, 278, 286, 322, 329, 348, 359], 'Bodycon Pink': [7, 98, 108, 152, 170, 222, 235, 340], 'Sheath Purple': [8], 'Sheath Beige': [9], 'Skater Multicolor': [10, 151, 156, 327, 334], 'Flare Green': [11,

[nltk_data] Downloading package stopwords to C:\Users\arnav
[nltk_data]     jain\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [19]:
dict_bigram_sorted= sorted(dict_bigram, key=dict_bigram.get, reverse=True)
print(dict_bigram_sorted)

[('Fit', 'Flare'), ('Bodycon', 'Black'), ('Bodycon', 'Green'), ('Flare', 'Purple'), ('Flare', 'Multicolor'), ('Sheath', 'Black'), ('Aline', 'Black'), ('Dark', 'Blue'), ('Flare', 'Black'), ('Flare', 'Blue'), ('Maxi', 'Black'), ('Dress', 'With'), ('Aline', 'Blue'), ('Bodycon', 'Pink'), ('Bodycon', 'Maroon'), ('Maxi', 'Multicolor'), ('Maxi', 'Pink'), ('Bodycon', 'Purple'), ('Flare', 'Pink'), ('Bodycon', 'Blue'), ('Flare', 'Green'), ('High', 'Low'), ('Gown', 'Brown'), ('Skater', 'Multicolor'), ('Maxi', 'Maroon'), ('Maxi', 'Yellow'), ('Flare', 'Yellow'), ('Aline', 'White'), ('Layered', 'Black'), ('Light', 'Blue'), ('Aline', 'Multicolor'), ('Maxi', 'White'), ('Maxi', 'Green'), ('Ethnic', 'Dress'), ('Bodycon', 'Brown'), ('Aline', 'Pink'), ('Maxi', 'Dark'), ('Aline', 'Purple'), ('Wrap', 'Black'), ('Aline', 'Yellow'), ('Flare', 'White'), ('Wrap', 'Pink'), ('Aline', 'Brown'), ('Multicolor', 'Dress'), ('Flare', 'Light'), ('T', 'Shirt'), ('Shirt', 'Grey'), ('Waist', 'Multicolor'), ('Gathered', 'Ma

In [20]:
row=dict_bigram_sorted[0:5]+dict_bigram_sorted[-5:]
print(row)

[('Fit', 'Flare'), ('Bodycon', 'Black'), ('Bodycon', 'Green'), ('Flare', 'Purple'), ('Flare', 'Multicolor'), ('Shirt', 'Blue'), ('Skater', 'Red'), ('Sheath', 'Maroon'), ('Pinafore', 'Dark'), ('Gathered', 'Dark')]


In [21]:
res = [] 
for el in row: 
#         sub = el.split(', ')
    sub=" ".join(el)
    res.append(list(sub.split("*"))) 
print(res)

[['Fit Flare'], ['Bodycon Black'], ['Bodycon Green'], ['Flare Purple'], ['Flare Multicolor'], ['Shirt Blue'], ['Skater Red'], ['Sheath Maroon'], ['Pinafore Dark'], ['Gathered Dark']]


In [22]:
import csv
with open('dress_top_bottom.csv', 'w', newline='') as file:
    res = [] 
    for el in row: 
#         sub = el.split(', ')
        sub=" ".join(el)
        res.append(list(sub.split("*"))) 

    for i in range(len(res)):
        res[i]=res[i]+item_bigram[res[i][0]]

    print(res)
    writer = csv.writer(file)
    writer.writerows(res)

[['Fit Flare', 0, 1, 2, 11, 12, 20, 25, 26, 27, 30, 32, 35, 37, 46, 49, 54, 55, 66, 69, 74, 76, 86, 91, 101, 105, 110, 111, 113, 130, 133, 135, 137, 146, 147, 148, 150, 157, 160, 165, 168, 174, 177, 183, 186, 209, 216, 219, 223, 224, 238, 250, 261, 263, 264, 265, 267, 276, 281, 284, 292, 293, 297, 300, 302, 312, 313, 314, 323, 331, 333, 338, 342, 343, 363, 364], ['Bodycon Black', 6, 18, 47, 52, 57, 62, 103, 106, 116, 136, 176, 178, 202, 207, 217, 220, 228, 249, 256, 258, 278, 286, 322, 329, 348, 359], ['Bodycon Green', 19, 24, 50, 75, 99, 132, 142, 182, 201, 203, 245, 269, 282, 318, 355], ['Flare Purple', 1, 37, 74, 110, 157, 160, 186, 223, 238, 261, 297, 312, 338], ['Flare Multicolor', 2, 12, 32, 66, 69, 86, 91, 147, 165, 267, 292, 300, 333], ['Shirt Blue', 347], ['Skater Red', 349], ['Sheath Maroon', 352], ['Pinafore Dark', 354], ['Gathered Dark', 360]]
